# OMERO Image Annotation Workflow with micro-SAM
This notebook guides you through annotating images in OMERO using the micro-SAM annotation plugin. Each step is explained clearly.

## 1. Setup and Installation
Install and import the required libraries. Make sure you have access to OMERO and the omero-annotate-ai package.

In [1]:
# Import the main package and check dependencies
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
 )

import os
from pathlib import Path

print(f"omero-annotate-ai version: {omero_annotate_ai.__version__}")
try:
    import ezomero
    print("OMERO functionality: Available")
except ImportError:
    print("OMERO functionality: Not available. Install with: pip install -e .[omero]")

omero-annotate-ai version: 0.1.0
OMERO functionality: Available


## 2. Connect to OMERO
Use the widget below to connect to your OMERO server. Fill in your server details and test the connection before proceeding.

In [ ]:
# Create and display the OMERO connection widget
print("OMERO Connection Setup")
conn_widget = create_omero_connection_widget()
conn_widget.display()
print("\nAfter connecting, run the next cell to continue.")

OMERO Connection Setup
Loaded configuration from connection history: root@localhost
Password loaded from keychain (no expiration)



After connecting, run the next cell to continue.


In [3]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established. Please use the widget above to connect.")

print("OMERO connection established.")

OMERO connection established.


## 3. Configure the Annotation Workflow
Set up your annotation workflow using the widget below. Select your working directory, OMERO container, and configure annotation parameters.

In [4]:
# Create and display the workflow widget
print("Annotation Workflow Setup")
workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()
print("\nAfter completing the workflow, run the next cell.")

Annotation Workflow Setup



After completing the workflow, run the next cell.


## 4. Review and Run the Annotation Pipeline
Preview your configuration, validate your OMERO selection, and run the annotation pipeline.

#### 4.1. Configuration summary

In [5]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

print("Configuration Summary:")
print(f"Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"Training Set: {config.name}")
print(f"Model: {config.ai_model.model_type}")
print(f"Channel: {config.spatial_coverage.channels}")
print(f"Output: {config.output.output_directory}")
print(f"Resume from Table: {config.workflow.resume_from_table}")
print(f"Read-only Mode: {config.workflow.read_only_mode}")

if config.processing.use_patches:
    print(f"Patches: {config.processing.patches_per_image} per image ({config.processing.patch_size[0]}×{config.processing.patch_size[1]})")

if config.spatial_coverage.three_d:
    print("3D processing: Enabled")

Configuration Summary:
Container: screen (ID: 1203)
Training Set: default_annotation_workflow
Model: vit_b_lm
Channel: [0]
Output: C:\Users\Maarten\omero_annotate_ai\omero_annotations2
Resume from Table: False
Read-only Mode: True
Patches: 1 per image (512×512)


#### 4.2. Convert configuration to an annotation pipeline

In [6]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"Validating {container_type} with ID {container_id}...")
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")
print(f"Found {container_type}: {container.getName()}")

Validating screen with ID 1203...
Found screen: idr0019-sero-nfkappab/screenA


#### 4.3. Run the annotation pipeline on the files
After loading of the files the napari will open for you to annotate the images

In [7]:
# Run the complete annotation pipeline using the updated full workflow function
print("Starting annotation pipeline...")
print(f"Using micro-SAM model: {config.ai_model.model_type}")

if config.processing.batch_size == 0:
    print("Processing: All images in one batch")
else:
    print(f"Processing: Batches of {config.processing.batch_size} images")

print("Napari will open for interactive annotation.")

try:
    # Run the complete micro-SAM workflow (setup, schema, table, annotation)
    table_id, updated_config = pipeline.run_full_micro_sam_workflow()
    print("Annotation pipeline completed successfully.")
    print(f"Tracking table ID: {table_id}")
    if updated_config.workflow.read_only_mode:
        print(f"Annotations saved locally to: {updated_config.output.output_directory}")
    else:
        print("Annotations uploaded to OMERO")
    print(f"Total images processed: {len(updated_config.get_processed())}")
except Exception as e:
    print(f"Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()

Starting annotation pipeline...
Using micro-SAM model: vit_b_lm
Processing: All images in one batch
Napari will open for interactive annotation.
Loading image IDs from screen 1203
Found 25872 image IDs
Loaded 7 images (by ID)
Defining annotation schema...
⚠️ Image smaller than patch size, using full image
⚠️ Image smaller than patch size, using full image
⚠️ Image smaller than patch size, using full image
⚠️ Image smaller than patch size, using full image
⚠️ Image smaller than patch size, using full image
⚠️ Image smaller than patch size, using full image
⚠️ Image smaller than patch size, using full image
Schema defined: 7 annotations
Read-only mode: Skipping OMERO table creation
Running micro-SAM annotation on 7 unprocessed annotations

🔄 Processing batch 1/1
   📦 Batch size: 7
   📊 Config state before: 0/7 processed
   🎨 Running annotation...
Loading 1 images using dask...
Materializing dask arrays to numpy...
   Processing chunk 1/1
Successfully loaded 1 images
Loaded 2D data shape:

Updating file 'vit_b_lm' from 'https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/diplomatic-bug/1.2/files/vit_b.pt' to 'C:\Users\Maarten\AppData\Local\micro_sam\micro_sam\Cache\models'.
100%|########################################| 375M/375M [00:00<00:00, 230GB/s]
Updating file 'vit_b_lm_decoder' from 'https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/diplomatic-bug/1.2/files/vit_b_decoder.pt' to 'C:\Users\Maarten\AppData\Local\micro_sam\micro_sam\Cache\models'.
100%|#####################################| 38.4M/38.4M [00:00<00:00, 18.9GB/s]
Precompute state for files: 100%|██████████| 7/7 [01:17<00:00, 11.07s/it]


Precomputation took 94.12706422805786 seconds (= 01:34 minutes)
The first image to annotate is image number 0
INFO: Computed embeddings for the 'Light Microscopy' model of size 'base'.
Loading next image: at index 1
Loading next image: at index 2
Loading next image: at index 3
Loading next image: at index 4
Loading next image: at index 5
Loading next image: at index 6
   ✅ Annotation completed, got results: <class 'dict'>
   🔄 Processing annotation results...
� Processing annotation results...
Found 7 annotation files for 7 metadata entries
📊 Updated 7/7 annotations
📈 Config now has 7/7 processed annotations
   📊 Config state after: 7/7 processed
   💾 Auto-saving config...
✅ Configuration saved to: C:\Users\Maarten\omero_annotate_ai\omero_annotations2\annotation_config.yaml
Config state saved to: C:\Users\Maarten\omero_annotate_ai\omero_annotations2\annotation_config.yaml
   ✅ Config saved
✅ Configuration saved to: C:\Users\Maarten\omero_annotate_ai\omero_annotations2\annotation_config

{
    _id = <nil>
    _details = object #1 (::omero::model::Details)
    {
        _owner = <nil>
        _group = <nil>
        _creationEvent = <nil>
        _updateEvent = <nil>
        _permissions = <nil>
        _externalInfo = <nil>
        _call = {}
        _event = <nil>
    }
    _loaded = True
    _version = <nil>
    _pixelsFileMapsSeq = 
    {
    }
    _pixelsFileMapsLoaded = True
    _pixelsFileMapsCountPerOwner = {}
    _path = object #2 (::omero::RString)
    {
        _val = C:\Users\Maarten\omero_annotate_ai\omero_annotations2
    }
    _repo = <nil>
    _size = object #3 (::omero::RLong)
    {
        _val = 5479
    }
    _atime = <nil>
    _mtime = <nil>
    _ctime = <nil>
    _hasher = object #4 (::omero::model::ChecksumAlgorithm)
    {
        _id = <nil>
        _details = object #5 (::omero::model::Details)
        {
            _owner = <nil>
            _group = <nil>
            _creationEvent = <nil>
            _updateEvent = <nil>
            _permissio

Error during annotation pipeline: exception ::omero::ApiUsageException
{
    serverStackTrace = ome.conditions.ApiUsageException: This instance is read-only
	at ome.security.basic.EventHandler.invoke(EventHandler.java:136)
	at org.springframework.aop.framework.ReflectiveMethodInvocation.proceed(ReflectiveMethodInvocation.java:179)
	at org.springframework.orm.hibernate3.HibernateInterceptor.invoke(HibernateInterceptor.java:119)
	at org.springframework.aop.framework.ReflectiveMethodInvocation.proceed(ReflectiveMethodInvocation.java:179)
	at org.springframework.transaction.interceptor.TransactionInterceptor$1.proceedWithInvocation(TransactionInterceptor.java:99)
	at org.springframework.transaction.interceptor.TransactionAspectSupport.invokeWithinTransaction(TransactionAspectSupport.java:282)
	at org.springframework.transaction.interceptor.TransactionInterceptor.invoke(TransactionInterceptor.java:96)
	at org.springframework.aop.framework.ReflectiveMethodInvocation.proceed(ReflectiveMethodI

Traceback (most recent call last):
  File "C:\Users\Maarten\AppData\Local\Temp\ipykernel_32884\3089686605.py", line 14, in <module>
    table_id, updated_config = pipeline.run_full_microsam_workflow()
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Maarten\Documents\Github\omero_annotate_ai\src\omero_annotate_ai\core\annotation_pipeline.py", line 1136, in run_full_microsam_workflow
    return self.run_microsam_annotation()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Maarten\Documents\Github\omero_annotate_ai\src\omero_annotate_ai\core\annotation_pipeline.py", line 1037, in run_microsam_annotation
    self._finalize_workflow(processed_count)
  File "C:\Users\Maarten\Documents\Github\omero_annotate_ai\src\omero_annotate_ai\core\annotation_pipeline.py", line 1007, in _finalize_workflow
    self._upload_annotation_config_to_omero()
  File "C:\Users\Maarten\Documents\Github\omero_annotate_ai\src\omero_annotate_ai\core\annotation_pipeline.

#### Save the configuration file to the local folder

In [ ]:
# Export configuration for future use
config_filename = f"annotation_config_{config.name}.yaml"
config_path = Path(config.output.output_directory) / config_filename

try:
    config.save_yaml(config_path)
    print(f"Configuration saved to: {config_path}")
except Exception as e:
    print("Could not save to output folder, saving to current directory")
    config.save_yaml(config_filename)
    print(f"Configuration saved to: {config_filename}")

print("To reuse this configuration:")
print("from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")

## 5. Cleanup (Recommended)
Close the OMERO connection when you are finished. This is good practice, especially when working with shared servers.

In [ ]:
# Close OMERO connection (optional, but recommended)
if 'conn' in locals() and conn is not None:
    conn.close()
    print("OMERO connection closed.")